# Pipeline for first classify and then predict


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, balanced_accuracy_score, confusion_matrix
import pickle 
from sklearn.model_selection import KFold
import geopandas as gpd
from mgwr.mgwr.gwr import MGWR, GWR
from mgwr.mgwr.sel_bw import Sel_BW
from sklearn.preprocessing import StandardScaler




In [34]:
#Split: 
with open('../../k_fold_investigate_errors.pkl', 'rb') as f:
    loaded_r2_scores_dict = pickle.load(f)
    loaded_grain_pred = pickle.load(f)

train_set_whole = pd.read_csv('../../data/train_set_with_feat_cleanCorr.csv')
train_set_class = train_set_whole


med_grain_pred = {keys: np.nanmedian(loaded_grain_pred[keys],axis=0) for keys in loaded_grain_pred.keys()}


diff_grain_size = {keys: train_set_class['mean_gs']-med_grain_pred[keys] for keys in  loaded_grain_pred.keys()}

key_to_watch = 20
error_thres = 5
idx_error = np.abs(diff_grain_size[key_to_watch])>=error_thres # True where we have outliers

idx_Corr = ~idx_error 
y_class = np.zeros(len(train_set_class))
y_class[idx_Corr] = 1
items_to_remove = ['id', 'mean_gs', 'sd', 'skewness', 'kurtosis', 'current_max', 'current_min','sample_type','y_im', 'x_im',  'x', 'y','comb_dist']

X_class = train_set_class.drop(items_to_remove, axis=1)


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.3, random_state=12)

# Apply SMOTE to handle the imbalanced dataset
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
#X_train_smote, y_train_smote = X_train, y_train 
# Define the classifiers to be tested
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Balanced Random Forest": BalancedRandomForestClassifier()
}

# Dictionary to store the F1 scores of each classifier
penalized_f1_scores = {}
conf_mat = {}
# Iterate over classifiers and calculate F1 score using cross-validation
for name, clf in classifiers.items():
    clf.fit(X_train_smote, y_train_smote)
    y_pred = clf.predict(X_test)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)


    penalized_f1 = balanced_accuracy

   
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    penalized_f1_scores[name] = tn

# Rank classifiers based on F1 score
ranked_classifiers = sorted(penalized_f1_scores.items(), key=lambda item: item[1], reverse=True)

# Print the ranked classifiers and their F1 scores
for name, score in ranked_classifiers:
    print(f"{name}: {score:.4f}")

c:\Users\arnehb\Work\git repos\GeoHab-2025\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM: 20.0000
KNN: 18.0000
Logistic Regression: 15.0000
Random Forest: 9.0000
Balanced Random Forest: 7.0000


In [35]:
#KNN best
clf  = BalancedRandomForestClassifier()

#Smote the whole shit
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_class, y_class)
print(X_train_smote.columns)
clf.fit(X_train_smote, y_train_smote)





Index(['current_mean', 'gebco', 'x_m', 'y_m', 'slope', 'aspect', 'rough',
       'bpi', 'shore_dist', 'island_dist', 'current_range'],
      dtype='object')


BalancedRandomForestClassifier()

In [36]:
#lets load new data
test_set_whole = pd.read_csv('../../data/test_set_with_feat_cleanCorr.csv')
items_to_remove = ['id', 'current_max', 'current_min','y_im', 'x_im',  'x', 'y','comb_dist']
X_test_class =  test_set_whole.drop(items_to_remove, axis=1)
X_test_class = X_test_class[X_train_smote.columns]

y_pred = clf.predict(X_test_class)
print(sum(y_pred))
print(len(y_pred)-sum(y_pred))


487.0
41.0


In [37]:
#Train group 0 and 

k = 10 
kf = KFold(n_splits=k, shuffle=True, random_state=42)


data = train_set_whole


posibble_preds = list(data)
items_to_remove = ['id', 'mean_gs', 'sd', 'skewness', 'kurtosis', 'current_max', 'current_min','sample_type','y_im', 'x_im', 'x_m', 'y_m', 'current_mean', 'current_range', 'gebco', 'x', 'y', 'island_dist', 'comb_dist']
#remove "base" as well
updated_list = [item for item in posibble_preds if item not in items_to_remove]

#data['current_range'] = data['current_max'] - data['current_min']



predictors = ['current_mean', 'current_range', 'gebco']
response = 'mean_gs'

## Train data
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['x'], data['y']))
gdf = gdf.set_crs(epsg=4326)  # Assuming WGS84
gdf = gdf.to_crs(epsg=32633)  # Convert to UTM zone 33N

data['x'] = gdf.geometry.x
data['y'] = gdf.geometry.y
print(data['x'])


test_gdf = gpd.GeoDataFrame(test_set_whole, geometry=gpd.points_from_xy(test_set_whole['x'], test_set_whole['y']))
test_gdf = test_gdf.set_crs(epsg=4326)
test_gdf = test_gdf.to_crs(epsg=32633)

test_set_whole['x'] = test_gdf.geometry.x
test_set_whole['y'] = test_gdf.geometry.y
coords_test_all = pd.DataFrame(test_set_whole[['x', 'y']])



import itertools


combinations = []
for r in range(0, len(updated_list) + 1):
    combinations.extend(itertools.combinations(updated_list, r))

# Add each combination to A
#pred_result_t = [predictors + list(comb) for comb in combinations]
updated_list.append('island_dist')

comb2 = []
for r in range(1,len(updated_list)+1):
    comb2.extend(itertools.combinations(updated_list, r))

combtot = combinations + comb2

#all_combinations = set(combinations_without_new + combinations_with_new)

pred_result_t = [predictors + list(comb) for comb in combtot]

# Remove duplicates from pred_result_t
pred_result = []
for comb in pred_result_t:
    if comb not in pred_result:
        pred_result.append(comb)


coords_all = data[['x', 'y']]



0      -5.036759e+06
1      -4.986116e+06
2      -4.986116e+06
3      -4.987333e+06
4      -4.987276e+06
            ...     
2105   -5.001167e+06
2106   -4.970671e+06
2107   -4.951021e+06
2108   -4.910076e+06
2109   -4.951217e+06
Name: x, Length: 2110, dtype: float64


In [38]:
## Dele opp:
train_set_0 = data.loc[idx_error]
train_set_1 = data.loc[~idx_error]
res_dict = dict()
gs_res = dict()


In [40]:
## Train and predict 0

pred = 9
data = train_set_0
predictorsN =  pred_result[pred]
print(predictorsN)
X = data[predictorsN]
coords = coords_all[idx_error]
coords_test = coords_test_all[y_pred==0]
X_test_0 = test_set_whole.loc[y_pred==0]

X_test_t = X_test_0[predictorsN]

y = data[response]
#y_test = test_data[response]
count = 0
r2_scores = []
mean_gs_all = np.zeros((k, len(test_set_whole)))

for train_index, test_index in kf.split(X):
    print(count)
    X_train_t= X.iloc[train_index]
    scaler = StandardScaler().fit(X_train_t)
    X_train = scaler.transform(X_train_t)
    X_test = scaler.transform(X_test_t)
    y_train= y.iloc[train_index]
    y_train = y_train.values.reshape(-1, 1)
    coords_train = coords.iloc[train_index]
    selector = Sel_BW(coords_train.values, y_train, X_train, multi=False) # This creates the bandwidths for different input features
    bws = selector.search(verbose=True, search_method='golden_section', max_iter=100) # This searches for the optimal bandwidth (fields of influence)
    mgwr_model = GWR(coords_train.values, y_train, X_train, bws)
    results = mgwr_model.fit() # This fits the model to the data
    scale = results.scale
    residuals = results.resid_response
    if len(X_test)>len(residuals):
        X_test_part1 = X_test[:len(residuals)]
        X_test_part2 = X_test[len(residuals):]
        
        mean_gs_pred_part1 = mgwr_model.predict(coords_test.values[:len(residuals)], X_test_part1, scale, residuals).predictions
        mean_gs_pred_part2 = mgwr_model.predict(coords_test.values[len(residuals):], X_test_part2, scale, residuals).predictions
        
        mean_gs_pred = np.concatenate((mean_gs_pred_part1, mean_gs_pred_part2), axis=0)
    else:
        mean_gs_pred = mgwr_model.predict(coords_test.values, X_test, scale, residuals).predictions
        
    t = np.zeros(len(test_set_whole)) + np.nan
    t[y_pred==0] = np.squeeze(mean_gs_pred)
    mean_gs_all[count,:] = t
    count += 1

gs_res[0] = mean_gs_all

['current_mean', 'current_range', 'gebco', 'slope', 'shore_dist']
0
Bandwidth:  70.0 , score:  563.33
Bandwidth:  80.0 , score:  559.56
Bandwidth:  87.0 , score:  558.04
Bandwidth:  91.0 , score:  557.53
Bandwidth:  94.0 , score:  557.26
Bandwidth:  95.0 , score:  557.22
Bandwidth:  96.0 , score:  557.22
1
Bandwidth:  70.0 , score:  569.46
Bandwidth:  80.0 , score:  563.76
Bandwidth:  87.0 , score:  561.25
Bandwidth:  91.0 , score:  560.09
Bandwidth:  94.0 , score:  559.27
Bandwidth:  95.0 , score:  558.98
Bandwidth:  96.0 , score:  558.87
Bandwidth:  97.0 , score:  558.72
2
Bandwidth:  70.0 , score:  570.27
Bandwidth:  80.0 , score:  566.36
Bandwidth:  87.0 , score:  565.02
Bandwidth:  91.0 , score:  564.59
Bandwidth:  94.0 , score:  564.28
Bandwidth:  95.0 , score:  564.20
Bandwidth:  96.0 , score:  564.20
Bandwidth:  97.0 , score:  564.14
3
Bandwidth:  70.0 , score:  564.07
Bandwidth:  80.0 , score:  560.63
Bandwidth:  87.0 , score:  558.97
Bandwidth:  91.0 , score:  558.97
Bandwidt

In [41]:
## Train and predict 1

pred = 9
data = train_set_1
predictorsN =  pred_result[pred]
print(predictorsN)
X = data[predictorsN]
coords = coords_all[~idx_error]
coords_test = coords_test_all[y_pred==1]
X_test_1 = test_set_whole.loc[y_pred==1]

X_test_t = X_test_1[predictorsN]

y = data[response]
#y_test = test_data[response]
count = 0
r2_scores = []
mean_gs_all = np.zeros((k, len(test_set_whole)))

for train_index, test_index in kf.split(X):
    print(count)
    X_train_t= X.iloc[train_index]
    scaler = StandardScaler().fit(X_train_t)
    X_train = scaler.transform(X_train_t)
    X_test = scaler.transform(X_test_t)
    y_train= y.iloc[train_index]
    y_train = y_train.values.reshape(-1, 1)
    coords_train = coords.iloc[train_index]
    selector = Sel_BW(coords_train.values, y_train, X_train, multi=False) # This creates the bandwidths for different input features
    bws = selector.search(verbose=True, search_method='golden_section', max_iter=100) # This searches for the optimal bandwidth (fields of influence)
    mgwr_model = GWR(coords_train.values, y_train, X_train, bws)
    results = mgwr_model.fit() # This fits the model to the data
    scale = results.scale
    residuals = results.resid_response
    mean_gs_pred = mgwr_model.predict(coords_test.values, X_test, scale, residuals).predictions
    t = np.zeros(len(test_set_whole)) + np.nan
    #org_idx = X_test_t.index[train_index].tolist()
    t[y_pred==1] = np.squeeze(mean_gs_pred)

    mean_gs_all[count,:] = t
    count += 1

gs_res[1] = mean_gs_all

['current_mean', 'current_range', 'gebco', 'slope', 'shore_dist']
0
Bandwidth:  720.0 , score:  7581.90
Bandwidth:  1132.0 , score:  7669.07
Bandwidth:  465.0 , score:  7509.06
Bandwidth:  307.0 , score:  7414.89
Bandwidth:  210.0 , score:  7356.75
Bandwidth:  149.0 , score:  7347.98
Bandwidth:  112.0 , score:  7365.95
Bandwidth:  173.0 , score:  7345.14
Bandwidth:  187.0 , score:  7347.91
Bandwidth:  164.0 , score:  7344.46
Bandwidth:  158.0 , score:  7345.12
Bandwidth:  167.0 , score:  7344.39
Bandwidth:  170.0 , score:  7345.04
Bandwidth:  166.0 , score:  7344.23
Bandwidth:  165.0 , score:  7344.51
1
Bandwidth:  720.0 , score:  7550.06
Bandwidth:  1133.0 , score:  7639.04
Bandwidth:  465.0 , score:  7477.08
Bandwidth:  307.0 , score:  7380.90
Bandwidth:  210.0 , score:  7324.39
Bandwidth:  149.0 , score:  7315.50
Bandwidth:  112.0 , score:  7329.08
Bandwidth:  173.0 , score:  7311.39
Bandwidth:  187.0 , score:  7314.74
Bandwidth:  164.0 , score:  7312.85
Bandwidth:  178.0 , score:  

In [23]:

grain_total = gs_res[1]

for j in range(len(gs_res[0])):
    grain_total[j][y_pred ==0] = gs_res[0][j][y_pred ==0]

In [26]:
med_grain_pred_group = np.nanmedian(grain_total,axis=0)
print(len(med_grain_pred_group))

528


In [29]:
test_set_whole['mean_gs'] = med_grain_pred_group
submission = test_set_whole[['id', 'mean_gs']]
submission.to_csv(f'../../data/Submissions/k_fold_feature_{pred}_k_10_median_svm_classifier.csv', index=False)